# Week 3 IBM-Coursera Data Science Capstone

This is the third submission of the Data Science capstone project.

## 1) Obtain the required Toronto Neighborhood Table. 

1.1. Download the required libraries and retrieve the wikipedia page.

In [1]:
import pandas as pd 
import requests
wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(wiki_url,'html.parser')

1.2. Retrieve the columns of the Wikipedia table as separated list.

In [2]:
postalCode = []
boroughs = []
neighborhoods = []

for row in soup.find('table').findAll('tr'):
    cells = row.find_all('td')
    if (len(cells)>0):
        postalCode.append(cells[0].text)
        boroughs.append(cells[1].text)
        neighborhoods.append(cells[2].text.rstrip('\n'))

1.3. Build the desired data frame

In [3]:
Toronto_table = [('PostalCode',postalCode),('Borough',boroughs),('Neighborhood',neighborhoods)]
Toronto_df = pd.DataFrame.from_dict(dict(Toronto_table))
Toronto_df.head()

,Borough,Neighborhood,PostalCode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


1.4. Drop the rows with the "Not Assigned" description.

In [4]:
Toronto_df_na = Toronto_df[Toronto_df.Borough != 'Not assigned'].reset_index(drop = True)
Toronto_df_na.head()

,Borough,Neighborhood,PostalCode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,Harbourfront,M5A
3,Downtown Toronto,Regent Park,M5A
4,North York,Lawrence Heights,M6A


1.5. Group the data by the Postal Code and the Borough.

In [5]:
Toronto_df_group = Toronto_df_na.groupby(['PostalCode','Borough'],as_index = False).agg(lambda x: ','.join(x))
Toronto_df_group.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


1.6. Change the the data point with the "Not assgined" in the Neighborhood section.

In [6]:
na_neighbor_rows = Toronto_df_group.Neighborhood == 'Not assigned'
Toronto_df_group.loc[na_neighbor_rows, 'Neighborhood'] = Toronto_df_group.loc[na_neighbor_rows,'Borough']
Toronto_df_group[na_neighbor_rows]

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Queen's Park


1.7. Check the dimensions of the data frame.

In [7]:
Toronto_df_cl = Toronto_df_group
Toronto_df_cl.shape

(103, 3)

## 2) Retrieving the Coordinates for the Toronto DataFrame

2.1 Import the required libraries for section 2 and section 3.

In [8]:
import numpy as np
from pandas.io.json import json_normalize
import json
from geopy.geocoders import Nominatim
import wget
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

print('Libraries imported!')

//anaconda/lib/python2.7/site-packages/folium/__init__.py:59: UserWarning: This version of folium is the last to support Python 2. Transition to Python 3 to be able to receive updates and fixes. Check out https://python3statement.org/ for more info.
  UserWarning


Libraries imported!


2.2. I decided to work with the csv file that contains the latitudes and longitudes of the neighborhoods. I downloaded the file and then read it using the pd.read_csv function. I hid this part of the code for security reasons.

2.3. I checked the dimensions of the data frame

In [10]:
print(cords.shape)
cords.head()

(103, 3)


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


2.4. I performed a join operation on the on the two data sets. 

In [11]:
Toronto_df_join = Toronto_df_cl.set_index('PostalCode')
cords_join = cords.set_index('Postal Code')
Toronto_df_cords = pd.concat([Toronto_df_join,cords_join], axis=1,join='inner')

In [12]:
Toronto_df_cords.index.name = 'PostalCode'
Toronto_df_cords.reset_index(inplace=True)

2.5. I checked the shape and the resulting data set

In [13]:
print(Toronto_df_cords.shape)
Toronto_df_cords.head()

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## 3) Clusterin Toronto Neighborhoods

3.1. Retrieve the latitude and longitude of Toronto

In [14]:
addr = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto-neigh")
location = geolocator.geocode(addr)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Toronto are 43.653963, -79.387207.


3.2. Plot the location of the neighborhoods on the map.

In [15]:
Toronto_map = folium.Map(location=[latitude,longitude],zoom_start=10.49)

for lat, long, post, borough, neigh in zip(Toronto_df_cords['Latitude'],Toronto_df_cords['Longitude'], Toronto_df_cords['PostalCode'],Toronto_df_cords['Borough'],Toronto_df_cords['Neighborhood']):
    label = "{} ({}): {}".format(borough,post,neigh)
    popup = folium.Popup(label,parse_html=True)
    folium.CircleMarker(
    [lat,long],
    radius=5,
    popup=popup,
    color='purple',
    fill = True,
    fill_color='purple',
    fill_opacity=0.7,
    parse_html=False).add_to(Toronto_map)

Toronto_map

3.3. I decided to work with the boroughs that contain Toronto within their name (East Toronto, Central Toronto, Downtown Toronto, West Toronto).

In [16]:
Toronto_boroughs = ['East Toronto', 'Central Toronto', 'Downtown Toronto', 'West Toronto']
Toronto_clust_df = Toronto_df_cords[Toronto_df_cords['Borough'].isin(Toronto_boroughs)].reset_index(drop=True)
print(Toronto_clust_df.shape)
Toronto_clust_df.head()

(38, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


3.4. Plot the location of the Neighborhoods that are located within the selected boroughs.

In [17]:
Toronto_map2 = folium.Map(location=[latitude, longitude], zoom_start=11.7)

for lat, long, post, borough, neigh in zip(Toronto_clust_df['Latitude'], Toronto_clust_df['Longitude'], Toronto_clust_df['PostalCode'], Toronto_clust_df['Borough'], Toronto_clust_df['Neighborhood']):
    label = "{} ({}): {}".format(borough, post, neigh)
    popup = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, long],
        radius=5,
        popup=popup,
        color='green',
        fill=True,
        fill_color='green',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map2)
    
Toronto_map2

3.4. Using my credentials I got access to the Foursquare API. I erased my credential, to be able to share it on Github

3.5. Get recommended places within the selected boroughs.

In [19]:
radius = 500
LIMIT = 100

venues = []

for lat, long, post, borough, neighborhood in zip(Toronto_clust_df['Latitude'], Toronto_clust_df['Longitude'], Toronto_clust_df['PostalCode'], Toronto_clust_df['Borough'], Toronto_clust_df['Neighborhood']):
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    for venue in results:
        venues.append((
            post, 
            borough,
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

3.6. Create a data frame of the found venues, check the dimensions of the data frame and get a look into the first data points.

In [20]:
venues_df = pd.DataFrame(venues)
venues_df.columns = ['PostalCode', 'Borough', 'Neighborhood', 'BoroughLatitude', 'BoroughLongitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']
print(venues_df.shape)
venues_df.head()

(1696, 9)


,PostalCode,Borough,Neighborhood,BoroughLatitude,BoroughLongitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
1,M4E,East Toronto,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
2,M4E,East Toronto,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,Pantheon,43.677621,-79.351434,Greek Restaurant


3.7. Group the data within the data frame based on the postal code, borough and neighborhood and check how many venues were obtained by postal code.

In [21]:
venues_df.groupby(['PostalCode', 'Borough', 'Neighborhood'])['VenueName'].count()

PostalCode  Borough           Neighborhood                                                                                        
M4E         East Toronto      The Beaches                                                                                               4
M4K         East Toronto      The Danforth West,Riverdale                                                                              42
M4L         East Toronto      The Beaches West,India Bazaar                                                                            18
M4M         East Toronto      Studio District                                                                                          38
M4N         Central Toronto   Lawrence Park                                                                                             3
M4P         Central Toronto   Davisville North                                                                                          9
M4R         Central Toronto   North Toron

3.8. Check how many categories of venues were returned.

In [22]:
len(venues_df['VenueCategory'].unique())

237

3.9. Perform one hot encoding to be able to analyze the venues per area.

In [23]:
# one hot encoding
Toronto_clust_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add postal, borough and neighborhood column back to dataframe
Toronto_clust_onehot['PostalCode'] = venues_df['PostalCode'] 
Toronto_clust_onehot['Borough'] = venues_df['Borough'] 
Toronto_clust_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move postal, borough and neighborhood column to the first column
fixed_columns = list(Toronto_clust_onehot.columns[-3:]) + list(Toronto_clust_onehot.columns[:-3])
Toronto_clust_onehot = Toronto_clust_onehot[fixed_columns]

print(Toronto_clust_onehot.shape)
Toronto_clust_onehot.head()

(1696, 240)


,PostalCode,Borough,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,M4E,East Toronto,The Beaches,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4K,East Toronto,"The Danforth West,Riverdale",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


3.10. Get the frequency of occurance of each category within an area.

In [24]:
Toronto_clust_venues_freq = Toronto_clust_onehot.groupby(['PostalCode', 'Borough', 'Neighborhoods']).mean().reset_index()
print(Toronto_clust_venues_freq.shape)
Toronto_clust_venues_freq.head()

(38, 240)


,PostalCode,Borough,Neighborhoods,Accessories Store,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,...,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M4E,East Toronto,The Beaches,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.25000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,M4K,East Toronto,"The Danforth West,Riverdale",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.02381,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023810
2,M4L,East Toronto,"The Beaches West,India Bazaar",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,M4M,East Toronto,Studio District,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026316
4,M4N,Central Toronto,Lawrence Park,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


3.11. Get the 10 most visited venue types in each area

In [25]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
areaColumns = ['PostalCode', 'Borough', 'Neighborhoods']
freqColumns = []
for ind in np.arange(num_top_venues):
    try:
        freqColumns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        freqColumns.append('{}th Most Common Venue'.format(ind+1))
columns = areaColumns+freqColumns
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['PostalCode'] = Toronto_clust_venues_freq['PostalCode']
neighborhoods_venues_sorted['Borough'] = Toronto_clust_venues_freq['Borough']
neighborhoods_venues_sorted['Neighborhoods'] = Toronto_clust_venues_freq['Neighborhoods']

for ind in np.arange(Toronto_clust_venues_freq.shape[0]):
    row_categories = Toronto_clust_venues_freq.iloc[ind, :].iloc[3:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    neighborhoods_venues_sorted.iloc[ind, 3:] = row_categories_sorted.index.values[0:num_top_venues]

neighborhoods_venues_sorted.sort_values(freqColumns, inplace=True)
neighborhoods_venues_sorted

,PostalCode,Borough,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Airport,Airport Food Court,Airport Gate
32,M6J,West Toronto,"Little Portugal,Trinity",Bar,Coffee Shop,Men's Store,Asian Restaurant,New American Restaurant,Cocktail Bar,Pizza Place,Bakery,Café,Vietnamese Restaurant
35,M6R,West Toronto,"Parkdale,Roncesvalles",Breakfast Spot,Gift Shop,Dog Run,Dessert Shop,Movie Theater,Bar,Bank,Eastern European Restaurant,Restaurant,Italian Restaurant
3,M4M,East Toronto,Studio District,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Gym,Brewery,Seafood Restaurant
30,M6G,Downtown Toronto,Christie,Café,Grocery Store,Park,Convenience Store,Diner,Nightclub,Restaurant,Italian Restaurant,Baby Store,Coffee Shop
25,M5S,Downtown Toronto,"Harbord,University of Toronto",Café,Italian Restaurant,Bookstore,Restaurant,Bar,Bakery,Japanese Restaurant,Dessert Shop,Sandwich Place,Beer Store
26,M5T,Downtown Toronto,"Chinatown,Grange Park,Kensington Market",Café,Vegetarian / Vegan Restaurant,Mexican Restaurant,Coffee Shop,Bar,Bakery,Dumpling Restaurant,Vietnamese Restaurant,Chinese Restaurant,Record Shop
13,M5A,Downtown Toronto,"Harbourfront,Regent Park",Coffee Shop,Bakery,Park,Pub,Mexican Restaurant,Breakfast Spot,Theater,Restaurant,Café,Chocolate Shop
33,M6K,West Toronto,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Breakfast Spot,Yoga Studio,Italian Restaurant,Convenience Store,Climbing Gym,Restaurant,Caribbean Restaurant,Burrito Place
15,M5C,Downtown Toronto,St. James Town,Coffee Shop,Café,Hotel,Restaurant,Clothing Store,Breakfast Spot,Bakery,Cocktail Bar,Gastropub,Cosmetics Shop


3.12. Perform K-means clustering to see how the neighborhoods have been grouped.

In [26]:
kclusters = 3

Toronto_freq_clustering = Toronto_clust_venues_freq.drop(['PostalCode', 'Borough', 'Neighborhoods'], 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_freq_clustering)

Toronto_clustered_df = Toronto_clust_df
Toronto_clustered_df['Cluster'] = kmeans.labels_

Toronto_clustered_df = Toronto_clustered_df.join(neighborhoods_venues_sorted.drop(['Borough', 'Neighborhoods'], 1).set_index('PostalCode'), on='PostalCode')
Toronto_clustered_df.sort_values(['Cluster'] + freqColumns, inplace=True)
Toronto_clustered_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,M5P,Central Toronto,"Forest Hill North,Forest Hill West",43.696948,-79.411307,0,Jewelry Store,Trail,Sushi Restaurant,Park,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Yoga Studio
10,M4W,Downtown Toronto,Rosedale,43.679563,-79.377529,0,Park,Playground,Trail,Discount Store,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
22,M5N,Central Toronto,Roselawn,43.711695,-79.416936,1,Ice Cream Shop,Garden,Yoga Studio,Dog Run,Filipino Restaurant,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant
27,M5V,Downtown Toronto,"CN Tower,Bathurst Quay,Island airport,Harbourf...",43.628947,-79.394420,2,Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Sculpture Garden,Harbor / Marina,Plane,Airport,Airport Food Court,Airport Gate
32,M6J,West Toronto,"Little Portugal,Trinity",43.647927,-79.419750,2,Bar,Coffee Shop,Men's Store,Asian Restaurant,New American Restaurant,Cocktail Bar,Pizza Place,Bakery,Café,Vietnamese Restaurant
35,M6R,West Toronto,"Parkdale,Roncesvalles",43.648960,-79.456325,2,Breakfast Spot,Gift Shop,Dog Run,Dessert Shop,Movie Theater,Bar,Bank,Eastern European Restaurant,Restaurant,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Gastropub,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Gym,Brewery,Seafood Restaurant
30,M6G,Downtown Toronto,Christie,43.669542,-79.422564,2,Café,Grocery Store,Park,Convenience Store,Diner,Nightclub,Restaurant,Italian Restaurant,Baby Store,Coffee Shop
25,M5S,Downtown Toronto,"Harbord,University of Toronto",43.662696,-79.400049,2,Café,Italian Restaurant,Bookstore,Restaurant,Bar,Bakery,Japanese Restaurant,Dessert Shop,Sandwich Place,Beer Store


3.13. Using the obtained data frame, display the clusters of neighborhoods within the Toronto map.

In [27]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, post, bor, poi, cluster in zip(Toronto_clustered_df['Latitude'], Toronto_clustered_df['Longitude'], Toronto_clustered_df['PostalCode'], Toronto_clustered_df['Borough'], Toronto_clustered_df['Neighborhood'], Toronto_clustered_df['Cluster']):
    label = folium.Popup('{} ({}): {} - Cluster {}'.format(bor, post, poi, cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Upon observing the clusters and the associated information, it is possible to infer that:

1. Cluster 0 -> The venues part of this cluster are parks, trails and playgrounds. Probably mainly a living area.
2. Cluster 1 -> It corresponds to the Roselawn neighborhood and it is mainly visited for recreation activities and restaurants.
3. Cluster 2 -> It is the largest cluster and the neighborhoods that are part of it have mainly coffe shops, restaurants and hotels. A closer inspection of the most visited venues reveal the presence of an airport and othe touristic venues. Probably the neighborhoods part of this cluster are within touristic areas or working areas.